In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# results for genetic correlation with disease
df_corr = pd.read_csv('results/rg_cvds.csv')
df_dis = pd.read_csv('gwas_data/cvd_gwas.csv')

df_corr = df_corr.merge(df_dis, left_on='p2', right_on='pheno', how='inner')
# SORT BY pheno_abv
df_corr = df_corr.sort_values(['p1', 'pheno_abbv'])

In [ ]:
# errorbars, color by subtype (p1)
diseases = df_corr['pheno_abbv'].unique()
print(diseases, len(diseases))
# positions for each disease, step 1.5, starting from 0
positions = np.arange(0, len(diseases) * 1.5, 1.5)
print(positions)

In [ ]:
def p_marker(p):
    if p < 0.05:
        return '*'
    else:
        return ''
df_corr['p_marker'] = df_corr['P_FDR'].apply(p_marker)

In [ ]:
import os
os.makedirs('results/figs', exist_ok=True)

In [ ]:
annot = True
# plot the results
fig, ax = plt.subplots(figsize=(10.5, 6))
# Iterate over subtypes to plot each one with its respective color, adjusted position, and error bars

# Adjust x positions to separate subtypes
adjusted_positions = [positions[diseases.tolist().index(disease)]
                      for disease in df_corr['pheno_abbv']]
# Plot error bars
ax.errorbar(adjusted_positions, df_corr['rg'], yerr=df_corr['se'],
            fmt='o', color='#0091DA', capsize=0,
            markersize=15, elinewidth=5, capthick=2)

# Annotate rg values
if annot:
    for i, (pos, row) in enumerate(zip(adjusted_positions, df_corr.iterrows())):
        _, row = row
        annot_text = f"{row['rg']:.3f}{row['p_marker']}"
        # if row['p_marker'] != '':
        ax.annotate(annot_text, (pos, row['rg'] + row['se'] + 0.015), textcoords="offset points",
                    xytext=(-1, 5), ha='center', color='black', fontsize=24)

# line y=0
ax.axhline(0, color='gray', linewidth=2, linestyle='--')

# Improve layout
ax.set_xticks(positions)
ax.set_xticklabels(diseases)
# ylabel: h2: Heritability
ax.set_ylabel('Genetic Correlation', fontsize=25)
ax.set_xlabel('Disease', fontsize=25)
# title
# ax.set_title('Genetic Correlation', fontsize=28)
ax.tick_params(axis='both', which='major', labelsize=24)
# x rotation 90
# plt.xticks(rotation=90)
# xlim
ax.set_xlim(-1, len(diseases) * 1.5 + 0.2)
ax.set_ylim(-0.2, 0.3)
# remove top and right spines
# sns.despine()
# remove grid
ax.grid(False)
# left and bottom spines, color black, width 2
for spine in ['left', 'bottom', 'top', 'right']:
    ax.spines[spine].set_linewidth(3)
    ax.spines[spine].set_color('black')

# add a text in the bottom '*', p < 0.05
if annot:
    plt.text(0.02, 0.06, '*, FDR Adjusted P-value < 0.05', fontsize=20, color='black', #  ha='center', va='center',
             transform=ax.transAxes, fontdict={'style': 'italic'})

plt.tight_layout()
if annot:
    plt.savefig('results/figs/genetic_corr_disease.jpg', dpi=300)
else:
    plt.savefig('results/figs/genetic_corr_disease.jpg', dpi=300)

In [ ]:
# change x and y axis labels
fig, ax = plt.subplots(figsize=(6, 8))
adjusted_positions = [positions[diseases.tolist().index(disease)] for disease in df_corr['pheno_abbv']]
ax.errorbar(df_corr['rg'], adjusted_positions, xerr=df_corr['se'],
            fmt='o', color='#0091DA', capsize=0,
            markersize=15, elinewidth=5, capthick=2)
# Annotate rg values
ax.set_xlabel('', fontsize=18)
ax.set_ylabel('', fontsize=18)
ax.set_title('', fontsize=28)
ax.set_yticks(positions)
ax.set_yticklabels(diseases, fontsize=24)
ax.tick_params(axis='both', which='major', labelsize=24)
ax.set_xlim(-0.3, 0.3)
sns.despine()
plt.grid(False)
plt.xticks(rotation=0)
# left and bottom spines, color black, width 2
for spine in ['left', 'bottom']:
    ax.spines[spine].set_linewidth(4)
    ax.spines[spine].set_color('black')
plt.tight_layout()
# set background color to white
plt.gcf().set_facecolor('white')
plt.savefig('results/figs/genetic_corr_disease_vertical.pdf', dpi=300, facecolor='white', bbox_inches='tight')